In [248]:
import os 
import pandas as pd
import numpy as np
import h5py
#import functions_read_data as rdat
# Tensorflow/Keras
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

DIR = "/tf/notebooks/schnemau/xAI_stroke_3d/"
os.chdir(DIR)
print(os.getcwd())
IMG_DIR = "/tf/notebooks/hezo/stroke_perfusion/data/"
OUTPUT_DIR = "/tf/notebooks/schnemau/xAI_stroke_3d/data/"
path_img = IMG_DIR + 'dicom_2d_192x192x3_clean_interpolated_18_02_2021_preprocessed2.h5'
path_tab = IMG_DIR + 'baseline_data_zurich_prepared.csv'


# Own functions
from functions.plot_slices import plot_slices
# ontram functions
from k_ontram_functions.ontram import ontram
from k_ontram_functions.ontram_loss import ontram_loss
from k_ontram_functions.ontram_metrics import ontram_acc, ontram_auc
from k_ontram_functions.ontram_predict import predict_ontram, get_parameters
from functions.augmentation3d import zoom, rotate, flip, shift

2.4.0
/tf/notebooks/schnemau/xAI_stroke_3d


In [249]:
which_split = 6
(X_train, X_valid, X_test, X_tab_train, X_tab_valid, X_tab_test), (Y_train, Y_valid, Y_test), results = read_and_split_img_data_andrea_maurice(
    path_img = IMG_DIR + 'dicom_2d_192x192x3_clean_interpolated_18_02_2021_preprocessed2.h5', 
    path_tab = IMG_DIR + 'baseline_data_zurich_prepared.csv', 
    path_splits = '/tf/notebooks/schnemau/xAI_stroke_3d/data/andrea_splits.csv', 
    split = which_split)

Y_train_MRS = to_categorical(Y_train)
Y_valid_MRS = to_categorical(Y_valid)
Y_test_MRS = to_categorical(Y_test)

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3], 1))
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1], X_valid.shape[2], X_valid.shape[3], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], X_test.shape[3], 1))

In [250]:
C = Y_train_MRS.shape[1]
batch_size = 5
epochs = 10
ntrain = 300
nvalid = 40
ntest = 40

# function for preprocessing
def train_preprocessing(data, label):
    """Process training data."""
    volume = data
    volume = zoom(volume)
    volume = rotate(volume)
    volume = shift(volume)
    volume = flip(volume)
    return volume, label

# Define data sets for Jonas Model - no ordinal structure of 0,1
train_loader = tf.data.Dataset.from_tensor_slices((X_train[:ntrain], Y_train[:ntrain]))
validation_loader = tf.data.Dataset.from_tensor_slices((X_valid[:nvalid], Y_valid[:nvalid]))
test_loader = tf.data.Dataset.from_tensor_slices((X_test[:ntest], Y_test[:ntest]))

train_dataset = (train_loader.shuffle(len(X_train[:ntrain]))
                 .map(train_preprocessing)
                 .batch(batch_size, drop_remainder = True))

validation_dataset = (validation_loader.batch(batch_size, drop_remainder = True))
test_dataset = (test_loader.batch(len(X_test[:ntest])))

# Define data sets MRS for ordinal structure
train_loader_MRS = tf.data.Dataset.from_tensor_slices((X_train[:ntrain], Y_train_MRS[:ntrain]))
validation_loader_MRS = tf.data.Dataset.from_tensor_slices((X_valid[:nvalid], Y_valid_MRS[:nvalid]))
test_loader_MRS = tf.data.Dataset.from_tensor_slices((X_test[:ntest], Y_test_MRS[:ntest]))

train_dataset_MRS = (train_loader_MRS.shuffle(len(X_train[:ntrain]))
                 .map(train_preprocessing)
                 .batch(batch_size, drop_remainder = True))

validation_dataset_MRS = (validation_loader_MRS.batch(batch_size, drop_remainder = True))
test_dataset_MRS = (test_loader_MRS.batch(len(X_test[:ntest])))

In [251]:
def img_model_linear(input_shape, output_shape, activation = "linear"):
    initializer = keras.initializers.he_normal(seed = 2202)
    in_ = keras.Input(shape = input_shape)
    x = keras.layers.Convolution3D(32, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu')(in_)
    x = keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = keras.layers.Convolution3D(32, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu')(x)
    x = keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = keras.layers.Convolution3D(64, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu')(x)
    x = keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = keras.layers.Convolution3D(64, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu')(x)
    x = keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(128, activation = 'relu')(x)
    x = keras.layers.Dropout(0.3)(x)
    x = keras.layers.Dense(128, activation = 'relu')(x)
    x = keras.layers.Dropout(0.3)(x)
    out_ = keras.layers.Dense(output_shape, activation = activation, use_bias = False)(x) 
    nn_im = keras.Model(inputs = in_, outputs = out_)
    return nn_im

In [252]:
ontram_1 = ontram(img_model_linear(X_train.shape[1:], 1))
ontram_2 = ontram(img_model_linear(X_train.shape[1:], 1))
ontram_3 = ontram(img_model_linear(X_train.shape[1:], 1))
ontram_4 = ontram(img_model_linear(X_train.shape[1:], 1))
ontram_5 = ontram(img_model_linear(X_train.shape[1:], 1))

ontram_1.compile(optimizer=keras.optimizers.Adam(),
                                    loss=ontram_loss(C, batch_size),
                                    metrics=[ontram_acc(C, batch_size)])

ontram_2.compile(optimizer=keras.optimizers.Adam(),
                                    loss=ontram_loss(C, batch_size),
                                    metrics=[ontram_acc(C, batch_size)])
ontram_3.compile(optimizer=keras.optimizers.Adam(),
                                    loss=ontram_loss(C, batch_size),
                                    metrics=[ontram_acc(C, batch_size)])

ontram_4.compile(optimizer=keras.optimizers.Adam(),
                                    loss=ontram_loss(C, batch_size),
                                    metrics=[ontram_acc(C, batch_size)])

ontram_5.compile(optimizer=keras.optimizers.Adam(),
                                    loss=ontram_loss(C, batch_size),
                                    metrics=[ontram_acc(C, batch_size)])

ontram_1.load_weights('/tf/notebooks/schnemau/xAI_stroke_3d/ensembling_results/model_ontram_CIB_0.h5')
ontram_2.load_weights('/tf/notebooks/schnemau/xAI_stroke_3d/ensembling_results/model_ontram_CIB_1.h5')
ontram_3.load_weights('/tf/notebooks/schnemau/xAI_stroke_3d/ensembling_results/model_ontram_CIB_2.h5')
ontram_4.load_weights('/tf/notebooks/schnemau/xAI_stroke_3d/ensembling_results/model_ontram_CIB_3.h5')
ontram_5.load_weights('/tf/notebooks/schnemau/xAI_stroke_3d/ensembling_results/model_ontram_CIB_4.h5')

In [294]:
sigmoid(ontram_1.predict(X_valid))

array([[0.8441382 ],
       [0.84616756],
       [0.84103656],
       [0.84914815],
       [0.85094553],
       [0.8498644 ],
       [0.8382138 ],
       [0.8537668 ],
       [0.84773195],
       [0.84653074],
       [0.8450489 ],
       [0.8390857 ],
       [0.8416159 ],
       [0.84795034],
       [0.8507792 ],
       [0.8489443 ],
       [0.84187025],
       [0.843887  ],
       [0.8480637 ],
       [0.8577904 ],
       [0.8340716 ],
       [0.85149276],
       [0.85094327],
       [0.83287317],
       [0.8504164 ],
       [0.8505758 ],
       [0.8331619 ],
       [0.84225655],
       [0.85278124],
       [0.83733946],
       [0.8504125 ],
       [0.8432271 ],
       [0.85069674],
       [0.843454  ],
       [0.8407243 ],
       [0.8540609 ],
       [0.8404932 ],
       [0.83605474],
       [0.8449385 ],
       [0.84095687],
       [0.8454236 ]], dtype=float32)

In [286]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def inv_sigmoid(x):
    return np.log(x / (1 - x))

In [276]:
predictions1 = ontram_1.predict(X_test)
predictions2 = ontram_2.predict(X_test)
predictions3 = ontram_3.predict(X_test)
predictions4 = ontram_4.predict(X_test)
predictions5 = ontram_5.predict(X_test)

In [290]:
stacked_predictions = np.stack((predictions1, predictions2, predictions3, predictions4, predictions5), axis=-1)
mean_predictions = np.mean(stacked_predictions, axis=-1)
sigmoid(mean_predictions)

array([[0.8193037 ],
       [0.8197383 ],
       [0.8185487 ],
       [0.8188418 ],
       [0.8207771 ],
       [0.82614714],
       [0.8232849 ],
       [0.8200314 ],
       [0.8238802 ],
       [0.8206554 ],
       [0.8126985 ],
       [0.8208447 ],
       [0.8163442 ],
       [0.8248391 ],
       [0.8201125 ],
       [0.8218048 ],
       [0.8254992 ],
       [0.82419246],
       [0.8223725 ],
       [0.8230573 ],
       [0.8130855 ],
       [0.82391715],
       [0.81903386],
       [0.81978595],
       [0.828861  ],
       [0.81848574],
       [0.81952757],
       [0.82667315],
       [0.8267326 ],
       [0.8230211 ],
       [0.82241046],
       [0.82126534],
       [0.8211114 ],
       [0.8286204 ],
       [0.8311095 ],
       [0.82293516],
       [0.82783055],
       [0.82295007],
       [0.8305129 ],
       [0.83465314],
       [0.8230796 ]], dtype=float32)

In [ ]:
h_l = []
    for model_name in model_names:
        cnn.load_weights(model_name)
        
        heatmap_prob_sum = np.zeros((volume.shape[0], volume.shape[1], volume.shape[2]), np.float32)
        heatmap_occ_n = np.zeros((volume.shape[0], volume.shape[1], volume.shape[2]), np.float32)

        # for n, (x, y, z, vol_float) in tqdm.tqdm(enumerate(iter_occlusion(volume, size = occlusion_size, stride = occlusion_stride))):
        #     X = vol_float.reshape(1, volume.shape[0], volume.shape[1], volume.shape[2], 1)
        #     out = model.predict(X)

        #     heatmap_prob_sum[x:x + occlusion_size[0], y:y + occlusion_size[1], z:z + occlusion_size[2]] += out[0]
        #     heatmap_occ_n[x:x + occlusion_size[0], y:y + occlusion_size[1], z:z + occlusion_size[2]] += 1

        ## Faster Implementation
        
        ## Generate all possible occlusions
        X = []
        xyz = []
        for n, (x, y, z, vol_float) in enumerate(iter_occlusion(
                volume, size = occlusion_size, stride = occlusion_stride)):
            X.append(vol_float.reshape(volume.shape[0], volume.shape[1], volume.shape[2], 1))
            xyz.append((x,y,z))
        
        X = np.array(X)
        out = cnn.predict(X) # do prediction for all occlusions at once 
        
        ## Add predictions to heatmap and count number of predictions per voxel
        for i in range(len(xyz)):
            x,y,z = xyz[i]
            heatmap_prob_sum[x:x + occlusion_size[0], y:y + occlusion_size[1], z:z + occlusion_size[2]] += out[i,0]
            heatmap_occ_n[x:x + occlusion_size[0], y:y + occlusion_size[1], z:z + occlusion_size[2]] += 1

        hm = heatmap_prob_sum / heatmap_occ_n # calculate average probability per voxel
        
        ## Get cutoff, invert heatmap if necessary and normalize
        cut_off = res_tab["y_pred_model_" + model_name[-4:-3]][0]
    
        if (res_tab["y_pred_class"][0] == 0 and invert_hm == "pred_class" and not both_directions) or (
            invert_hm == "never" and not both_directions): 
            hm[hm < cut_off] = cut_off
        elif (res_tab["y_pred_class"][0] == 1 and invert_hm == "pred_class" and not both_directions) or (
            invert_hm == "always" and not both_directions):
            hm[hm > cut_off] = cut_off
        elif both_directions:
            hm = hm - cut_off
        
        if normalize and not both_directions:
            hm = ((hm - hm.min())/(hm.max()-hm.min()))
        elif normalize and both_directions:
            hm_min_max = [np.min(hm), np.max(hm)]
            hm_abs_max = np.max(np.abs(hm_min_max))
            hm = hm / hm_abs_max
        
        h_l.append(hm)

## Trying to average, done wrongly

In [185]:
models = [get_parameters(ontram_1)['intercept'], 
          get_parameters(ontram_2)['intercept'], 
          get_parameters(ontram_3)['intercept'], 
          get_parameters(ontram_4)['intercept'], 
          get_parameters(ontram_5)['intercept']]

new_model = []
for index_layer in range(len(models[0])):
    current_weights = []
    for index_weights in range(len(models[0][index_layer])):

        current_weights.append(sum([models[index_models][index_layer][index_weights]                            
                                for index_models in range(len(models))])/len(models))
    
    new_model.append(current_weights)

In [216]:
ontram_ensemble = ontram(img_model_linear(X_train.shape[1:], 1))
ontram_ensemble.compile(optimizer=keras.optimizers.Adam(),
                                    loss=ontram_loss(C, batch_size),
                                    metrics=[ontram_acc(C, batch_size)])

In [217]:
for index, layer in enumerate(ontram_ensemble.mod_baseline.layers):
    if len(new_model[index]) == 0:
        weights = np.array([])
    else: 
        weights = new_model[index]
    
    ontram_ensemble.mod_baseline.layers[index].set_weights(weights)